In [ ]:
!pip install optuna

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import library
Pertama kami akan mengimport library dasar seperti numpy, pandas, matplotlib, dsb untuk memenuhi kebutuhan saat pre-processing ataupun modeling. Selain itu library pytorch akan digunakan untuk modelling, sesuai dengan materi perkuliahan Deep Learning yang kami dapatkan. 

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import time
import copy
import seaborn as sn
import seaborn as sns
import warnings as ws
ws.filterwarnings('ignore')
#--------------------------------------------------------
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import transforms,models
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm_notebook
from IPython.core.display import HTML,display
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report

import optuna
random_seed = 123
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
import numpy as np
np.random.seed(random_seed)

### Checking GPU Availability
Secara default, tensor diproses di CPU dan model juga diinisialisasi pada CPU. Oleh karena itu, kita harus secara manual memastikan bahwa operasi dilakukan dengan menggunakan GPU.

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print("Using GPU")
else:
    device = torch.device('cpu')
    print("Using CPU")

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Dataset/Vegetable Images.zip') as zipObj:
  zipObj.extractall()

### Walk through Input Directory
Kami menggunkan metode os.walk untuk membaca file dataset secara berulang kali

In [ ]:
def walk_through_dir(directory_name):
    for dirpaths,dirnames,filenames in os.walk(directory_name):
        text=f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpaths}'"
        print(text)

In [ ]:
walk_through_dir('/content/Vegetable Images')

Dataset vegetable image sebenarnya terdiri dari 15 kelas, yaitu bean, bitter gourd, bottle gourd, brinjal, broccoli, cabbage, capsicum, carrot, cauliflower, cucumber, papaya, potato, pumpkin, radish dan tomato. Gambar pada dataset ini berukuran sama yaitu 224 x 224 pixel berformat .jpg.
Namun sesuai petunjuk soal, data yang akan digunakan adalah **carrot**, **papaya**, dan **potato**.
Dari output di atas dapat dilihat untuk jumlah images pada masing-masing path. Terdapat masing-masing 1000 images untuk kelas carrot, papaya, potato pada data train, dan terdapat masing-masing 200 images untuk kelas carrot, papaya, potato pada data validation dan data test.

Menginisialisasi nama variable untuk direktori data train, data validasi dan data testing, serta pendefinisikan show_image function untuk melihat sampel data carrot, papaya dan potato pada data train dan data validation serta size dari imagesnya.

In [ ]:
train_dir='/content/Vegetable Images/train'
val_dir='/content/Vegetable Images/validation'
test_dir='/content/Vegetable Images/test'

In [ ]:
image_formats = ["png", "jpg"];

def show_images(image_files,name): 
    display(HTML('<H5 style="color:blue"> <b>Sampel Data {} </b></H5><hr>'.format(name)))
    fig = plt.figure(figsize=(10,10))
    fig.patch.set_facecolor('xkcd:white')
    for i in range(len(image_files)):
        plt.subplot(3,3,i+1)    
        img=mpimg.imread(image_files[i])
        plt.imshow(img)
        plt.tight_layout()
        plt.axis('off')
        plt.title(image_files[i].split("/")[5]+"\n"+"{}x{}".format(img.shape[0], img.shape[1])) # nama sayur dan ukuran gambar
    plt.show()

def list_files(dir):
    arr = []
    for root, dirs, files in os.walk(dir):
            for name in files:
                if name.endswith(".jpg") or name.endswith(".png"):
                    arr.append(os.path.join(root, name))
                    break
    return arr

### Sampel Data

Berikut adalah sampel data untuk masing-masing kelas (**carrot**, **Papaya**, dan **Potato**) pada data train, validation, dan data test.

In [ ]:
show_images(list_files(train_dir), "Train")
show_images(list_files(val_dir), "Validation")
show_images(list_files(test_dir), "Test")


### Data Preparation

#### Transformation (Load data to pytorch tensor)
Disini kami melakukan beberapa transformasi 
pada gambar seperti secara random memutar gambar secara horizontal, secara random memutar gambar dengan angle 15 derajat, ataupun melakukan GaussianBlur pada gambar dengan kernel size 5,9 dan lainnya.

In [ ]:
mean=[0.485,0.456,0.406]
std=[0.229,0.224,0.225]

transform_ = transforms.Compose([transforms.RandomHorizontalFlip(p=0.5),
                                 transforms.RandomRotation(15),
                                 transforms.ColorJitter(brightness=0.2,contrast=0.1,hue=0.1,saturation=0.1),
                                 transforms.RandomAffine(degrees=15, translate=(0.1,0.1), scale=(1, 2), shear=15),
                                 transforms.GaussianBlur(kernel_size=(5,9)),
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize(mean,std)])

##### Convert to Dataset

In [ ]:
train_dataset=torchvision.datasets.ImageFolder(root=train_dir,transform=transform_)
val_dataset=torchvision.datasets.ImageFolder(root=val_dir,transform=transform_)
test_dataset=torchvision.datasets.ImageFolder(root=test_dir,transform=transform_)

##### Defining Data Loader
Data Loader merupakan inti dari perangkat pemrosesan data di PyTorch untuk mempersiapkan data termasuk berbagai metode sampling, komputasi paralel, dan pemrosesan terdistribusi. Disini kami memberikan nilai batch size = 32, dimana akan  dalam 1 batch akan diload sebanyak 32 samples


In [ ]:
batch_size=32
train_dl=DataLoader(train_dataset,batch_size,shuffle=True)
val_dl=DataLoader(val_dataset,batch_size,shuffle=True)
test_dl=DataLoader(test_dataset,batch_size,shuffle=False)

In [ ]:
dataset_sizes = {'train':len(train_dl.dataset),'valid':len(val_dl.dataset)}
dataloaders = {'train':train_dl,'valid':test_dl}

In [ ]:
images, labels = next(iter(train_dl))
print("images-size:", images.shape)

# Modelling

## A. Resnet18 Pretained Models

Dengan menggunakan ResNet18 pretained model dengan PyTorch. Layer terakhir diubah agar sesuai dengan jumlah kelas yang digunakan yaitu 3 kelas (carrot, papaya, dan potato)

In [ ]:
resnet18_tf = models.resnet18(pretrained=True)
resnet18_tf = resnet18_tf.cuda() if device else resnet18_tf

In [ ]:
print(f'Resnet18 model summary:\n{resnet18_tf.named_parameters}')

Selanjutnya dilakukan setting parameter pada learning rate, loss function, dan optimizer. Learning rate yang digunakan adalah 0.001, loss function yang digunakan adalah Cross Entropy loss, dan optimizer yang digunakan adalah SGD.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_tf.parameters(), lr=0.0001, momentum=0.9)

def accuracy(out, labels):
    _,pred = torch.max(out, dim=1)
    return torch.sum(pred==labels).item()

num_ftrs = resnet18_tf.fc.in_features
resnet18_tf.fc = nn.Linear(num_ftrs, 3)
resnet18_tf.fc = resnet18_tf.fc.cuda()

Selanjutnya dengan epoch=10 dilakukan proses training model untuk memperoleh best validation accuracy.

In [ ]:
n_epochs = 10
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_dl)*3
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_dl):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        outputs = resnet18_tf(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        resnet18_tf.eval()
        for data_t, target_t in (val_dl):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = resnet18_tf(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(val_dl))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')
        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(resnet18_tf.state_dict(), 'resnet_tf.pt')
            print('Improvement-Detected, save-model')
    resnet18_tf.train()

In [ ]:
def plot_learning_curves(train_acc, val_acc, train_loss, val_loss):
    fig = plt.figure(figsize=(20, 6))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(train_acc, '-o',label='Train Acc.')
    ax.plot(val_acc, '--<', label='Validation Acc.')
    ax.set_title("Train-Validation Accuracy",size=12)
    ax.legend(loc='best',fontsize=12)
    ax.set_xlabel('Epoch', size=12)
    ax.set_ylabel('Accuracy', size=12)
    ax = fig.add_subplot(1, 2, 2)
    ax.plot(train_loss,'-o', label='Train Loss')
    ax.plot(val_loss,'--<', label='Validation Loss')
    ax.set_title("Train-Validation Loss",size=12)
    ax.legend(loc='best',fontsize=12)
    ax.set_xlabel('Epoch', size=12)
    ax.set_ylabel('Loss', size=12)
    plt.show()

In [ ]:
plot_learning_curves(train_acc, val_acc, train_loss, val_loss)

### Model Evaluation

Selanjutnya akan ditampilkan evaluasi model melalui confusion matrix dan classification report dengan menggunakan eval_model function.

In [ ]:
def eval_model(model):
    y_act = []
    y_pred= []
    model.eval()
    for data_t, target_t in (val_dl):
        data_t, target_t = data_t.to(device), target_t.to(device)
        outputs_t = model(data_t)
        _,pred_t = torch.max(outputs_t, dim=1)
        predictions = pred_t.to("cpu")
        y_pred.extend(predictions.numpy())
        actual = target_t.to("cpu")
        y_act.extend(actual.numpy())
    cf_matrix = confusion_matrix(y_act,y_pred)
    sns.set_theme(rc={'figure.figsize':(8,8)})
    ax = sns.heatmap(cf_matrix,annot=True,cmap='Reds',fmt="g",xticklabels=train_dataset.classes,yticklabels=train_dataset.classes,cbar=False)
    ax.set_ylabel('True Labels')
    ax.set_xlabel('Predicted Labels');
    plt.show()
    print(classification_report(y_act,y_pred))

In [ ]:
eval_model(resnet18_tf)

Dari output di atas dapat dilihat bahwa dari total 600 images pada data testing, sebanyak 593 images diklasifikasikan dengan tepat dan sisanya yaitu sebanyak 7 images mengalami misklasifikasi. Dari 7 image yang salah diklasikasikan tersebut, 3 images carrots diprediksi model merupakan image potatos,  2 images papayas diprediksi model merupakan image potatos, 1 images potatos diprediksi model merupakan image potatos, dan 1 images potatos diprediksi model merupakan image papayas. Dari confusion matrix tersebut, diperoleh akurasi, recall, dan precision masing-masing adalah 98%, 99%, dan 98%.

## Tuning parameters Resnet18 Pretained Model

Selanjutnya pada resnet18 pretained model, akan dilakukan tuning parameter terhadap learning rate, optimizer, dan loss function. Untuk melakukan hal tersebut, dibuat objective function untuk menentukan parameter yang akan dituning, memanggil train_model function dan mengembalikan skor evaluasinya (Objective value). Adapun library yg digunakan untuk tuning parameter adalah library optuna. Di dalam Objective Function, perlu mendefinisikan parameter yang ingin dioptimalkan. Dalam hal ini parameter yang dituning adalah :

- learning rate : dari 1e-4 hingga 1e-2.
- optimizer : SGD, Adam, dan Adadelta
- Loss Function : CrossEntropyLoss dan NLLLoss

In [ ]:
def objective(trial):
    
    # Hyperparameters we want optimize
    params = {
        "loss_function": trial.suggest_categorical('loss_function',["nn.CrossEntropyLoss()","nn.NLLLoss()"]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),
        "optimizer_name": trial.suggest_categorical('optimizer_name',['SGD', 'Adam', 'Adadelta'])
    }
    
    # Get pretrained model
    model = resnet18_tf
    
    # Define criterion
    criterion = eval(params['loss_function'])
    
    # Configure optimizer
    optimizer = getattr(
        torch.optim, params["optimizer_name"]
    )(model.parameters(), lr=params["learning_rate"])
    
    # Train model
    best_model, best_acc = train_model(trial, model, criterion, optimizer, num_epochs=10)
    
    # Save best model for each trial
    torch.save(best_model.state_dict(), f"model_tf_trial_{trial.number}.pth")
    
    return best_acc

In [ ]:
def train_model(trial, model, criterion, optimizer, num_epochs):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0


    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() 
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()
        
        trial.report(epoch_acc, epoch)
        if trial.should_prune():
            raise optuna.TrialPruned()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, best_acc

Untuk melakukan optimasi pada objective function, dicreate new study function yang sudah disediakan library optuna. untuk sampler menggunakan TPE sampler, pruner menggunakan MedianPruner untuk interrupt unpromising trialsnya, direction menggunakan maximize untuk memaksimalkan akurasi, dan n_trials adalah jumlah trials yang digunakan yaitu sebanyak 10.

In [ ]:
sampler = optuna.samplers.TPESampler()    
study = optuna.create_study(sampler=sampler, pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5, interval_steps=5), direction='maximize')
study.optimize(func=objective, n_trials=10)

Dari output diatas dapat dilihat bahwa akurasi yang diperoleh adalah 99,67% pada trial ke 1 dengan best parameters :

- loss_function : nn.CrossEntropyLoss()
- learning_rate : 0.0028325557834021796
- optimizer_name : Adadelta.



In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
optuna.visualization.plot_contour(study, params=['learning_rate','optimizer_name','loss_function'])

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

Berikut adalah best parameter dan best trial yang diperoleh, seperti yang dijelaskan sebelumnya.

In [ ]:
print("Best Parameters :", study.best_params)
print("Best Best Trial Number :", study.best_trial._number)

In [ ]:
resnet18_tf_tuned=resnet18_tf
resnet18_tf_tuned.load_state_dict(torch.load(f'model_tf_trial_{study.best_trial._number}.pth'))

Dengan menggunakan best model yaitu dengan parameter {'loss_function': 'nn.CrossEntropyLoss()', 'learning_rate': 0.0028325557834021796, 'optimizer_name': 'Adadelta'} diperoleh confusion matrix dan classification report sebagai berikut.

In [ ]:
eval_model(resnet18_tf_tuned)

Dari output di atas dapat dilihat bahwa dari total 600 images pada data testing, semua 600 images diklasifikasikan dengan tepat. Maka dari confusion matrix tersebut, diperoleh akurasi, recall, dan precision masing-masing adalah  100%.

## B. Resnet18 from Scratch

In [ ]:
class Block(nn.Module):
    
    def __init__(self, in_channels, out_channels, identity_downsample=None, stride=1):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.identity_downsample = identity_downsample
        
    def forward(self, x):
        identity = x
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)
        x += identity
        x = self.relu(x)
        return x

In [ ]:
class ResNet_18(nn.Module):
    
    def __init__(self, image_channels, num_classes):
        
        super(ResNet_18, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv2d(image_channels, 32, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        #resnet layers
        self.layer1 = self.__make_layer(32, 32, stride=1)
        self.layer2 = self.__make_layer(32, 64, stride=2)
        self.layer3 = self.__make_layer(64, 128, stride=2)
        self.layer4 = self.__make_layer(128, 256, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)
        
    def __make_layer(self, in_channels, out_channels, stride):
        
        identity_downsample = None
        if stride != 1:
            identity_downsample = self.identity_downsample(in_channels, out_channels)
            
        return nn.Sequential(
            Block(in_channels, out_channels, identity_downsample=identity_downsample, stride=stride), 
            Block(out_channels, out_channels)
        )
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x 
    
    def identity_downsample(self, in_channels, out_channels):
        
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=2, padding=1), 
            nn.BatchNorm2d(out_channels)
        )

In [ ]:
net_scracth = ResNet_18(3, 3)

In [ ]:
#count trainable parameters of the model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(net_scracth)

In [ ]:
#move the model to the device
net_scracth.to(device)
next(net_scracth.parameters()).is_cuda

In [ ]:
resnet18_scracths

Mendefinisikan nilai epoch = 10 dengan menggunakan CrossEntropyLoss Function dan Optimizer Adam


In [ ]:
#define everything we need for training
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net_scracth.parameters(), lr=0.0001, weight_decay=1e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

In [ ]:
n_epochs = 10
print_every = 10
valid_loss_min = np.Inf
val_loss = []
val_acc = []
train_loss = []
train_acc = []
total_step = len(train_dl)*3
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total=0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(train_dl):
        data_, target_ = data_.to(device), target_.to(device)
        optimizer.zero_grad()
        outputs = net_scracth(data_)
        loss = criterion(outputs, target_)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _,pred = torch.max(outputs, dim=1)
        correct += torch.sum(pred==target_).item()
        total += target_.size(0)
        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f}, train-acc: {(100 * correct/total):.4f}')
    batch_loss = 0
    total_t=0
    correct_t=0
    with torch.no_grad():
        net_scracth.eval()
        for data_t, target_t in (val_dl):
            data_t, target_t = data_t.to(device), target_t.to(device)
            outputs_t = net_scracth(data_t)
            loss_t = criterion(outputs_t, target_t)
            batch_loss += loss_t.item()
            _,pred_t = torch.max(outputs_t, dim=1)
            correct_t += torch.sum(pred_t==target_t).item()
            total_t += target_t.size(0)
        val_acc.append(100 * correct_t/total_t)
        val_loss.append(batch_loss/len(val_dl))
        network_learned = batch_loss < valid_loss_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t/total_t):.4f}\n')
        
        if network_learned:
            valid_loss_min = batch_loss
            torch.save(net_scracth.state_dict(), 'resnet_sc.pt')
            print('Improvement-Detected, save-model')
    net_scracth.train()

In [ ]:
plot_learning_curves(train_acc, val_acc, train_loss, val_loss)

In [ ]:
eval_model(net_scracth)

## Tuning parameters Resnet18 From Scratch

Seperti tuning parameter pada resnet18 pretained model sebelumnya, tuning parameter juga akan dilakukan pada resnet18 (scratch) terhadap learning rate, optimizer, dan loss function. Untuk melakukan hal tersebut, dibuat objective function untuk menentukan parameter yang akan dituning, memanggil train_model function yang sudah digenerate pada bagian sebelumnya dan mengembalikan skor evaluasinya (Objektive value/akurasi). Adapun library yg digunakan untuk tuning parameter adalah library optuna. Di dalam Objective Function, perlu mendefinisikan parameter yang ingin dioptimalkan. Dalam hal ini parameter yang dituning adalah :

- learning rate : dari 1e-4 hingga 1e-2.
- optimizer : SGD, Adam, dan Adadelta
- Loss Function : CrossEntropyLoss dan NLLLoss

In [ ]:
def objective_v2(trial):
    
    # Hyperparameters we want optimize
    params = {
        "loss_function": trial.suggest_categorical('loss_function',["nn.CrossEntropyLoss()","nn.NLLLoss()"]),
        "learning_rate": trial.suggest_loguniform('learning_rate', 1e-4, 1e-2),
        "optimizer_name": trial.suggest_categorical('optimizer_name',['SGD', 'Adam', 'Adadelta'])
    }
    
    model = net_scracth
    # Define criterion
    criterion = eval(params['loss_function'])
    
    # Configure optimizer
    optimizer = getattr(
        torch.optim, params["optimizer_name"]
    )(model.parameters(), lr=params["learning_rate"])
    
    # Train model
    best_model, best_acc = train_model(trial, model, criterion, optimizer, num_epochs=10)
    
    # Save best model for each trial
    torch.save(best_model.state_dict(), f"model_scr_trial_{trial.number}.pth")
    
    return best_acc

In [ ]:
sampler = optuna.samplers.TPESampler()    
study_scratch = optuna.create_study(
    sampler=sampler,
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=5, n_warmup_steps=5, interval_steps=5
    ),
    direction='maximize')
study_scratch.optimize(func=objective_v2, n_trials=10)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study_scratch)

In [ ]:
optuna.visualization.plot_contour(study_scratch, params=['learning_rate','optimizer_name','loss_function'])

In [ ]:
optuna.visualization.plot_slice(study_scratch)

In [ ]:
optuna.visualization.plot_param_importances(study_scratch)

In [ ]:
optuna.visualization.plot_optimization_history(study_scratch)

In [ ]:
print("Best Parameters :", study_scratch.best_params)
print("Best Best Trial Number :", study_scratch.best_trial._number)

In [ ]:
net_scracth_tuned=net_scracth
net_scracth_tuned.load_state_dict(torch.load(f'model_scr_trial_{study_scratch.best_trial._number}.pth'))

In [ ]:
eval_model(net_scracth_tuned)